In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

C:\Users\msubr\Anaconda3\envs\bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("UCF-crime-frames-descriptions-partitions.csv")

In [3]:
data.head(2)

,Unnamed: 0,frame,directory,class,Description,partition
0,0,Abuse001_x264.mp4_frame_000000.jpg,Abuse_frames,Normal,a man is standing in the middle of a room,train
1,1,Abuse001_x264.mp4_frame_000010.jpg,Abuse_frames,Normal,a man is standing in the middle of a room,train


In [4]:
train = data.loc[data["partition"] == "train"]
val = data.loc[data["partition"] == "validation"]
test = data.loc[data["partition"] == "test"]

In [5]:
labels = {"Normal": 0,
          "Abuse": 1,
          "Arrest": 2,
          "Arson": 3,
          "Burglary": 4,
          "Explosion": 5,
          "Fighting": 6,
          "RoadAccidents": 7,
          "Shooting": 8,
          "Vandalism": 9
         }

In [6]:
def evaluate(data):
    with torch.no_grad():
        d_pred = torch.zeros([len(data), 10])
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            batch = batch.to(device)
            pred = model(batch)
            d_pred[i:i+batch_size] = pred 
    return d_pred

In [12]:
def get_data(df):
    X = np.zeros((len(df), 1408))
    path = "d:/data/UCF-crime/Anomaly-Videos-frames-features"
    c = 0
    for _, f in df.iterrows():
        directory = f["directory"][:-7]
        X[c] = np.load(f"{path}/{directory}/{f['frame']}.npy")
        c += 1
    X = torch.tensor(X, dtype=torch.float32)
    y = list(df["class"])
    y = [labels[lbl] for lbl in y]
    y = torch.tensor(y, dtype=torch.float32)
    return X, y

In [8]:
def get_acc(pred, target, return_sum=False):
    s = (torch.argmax(pred, dim=1).round() == target).sum()
    if (return_sum == True):
        return s
    return s/len(pred)

In [10]:
def get_acc_per_lbl(pred, target, return_sum=False):
    for i in range(len(labels)):
        p = (target == i).sum()
        t = (torch.argmax(pred, dim=1) == i).sum()
        s = ((torch.argmax(pred, dim=1) == i) & (target == i)).sum()
        presc = s/t
        recall = s/p
        print(f"{i}, presc {s/t:.2f}, recall {s/p:.2f}, f1 {2*presc*recall/(presc+recall):.2f}\t total {p} {list(labels.keys())[i]}")
    
    return 

In [13]:
V, y_val = get_data(val)
V.shape, y_val.shape

(torch.Size([19330, 1408]), torch.Size([19330]))

In [14]:
X, y = get_data(train)
X.shape, y.shape

(torch.Size([124145, 1408]), torch.Size([124145]))

In [88]:
np.save( "train.npy", X.numpy())

In [89]:
np.save( "val.npy", V.numpy())

In [162]:
mean, m, M = torch.mean(X), torch.min(X), torch.max(X)

In [166]:
X = (X-mean)/(M-m)

In [173]:
V = (V-mean)/(M-m)

In [167]:
torch.mean(X), torch.max(X), torch.min(X)

(tensor(2.0759e-10), tensor(0.4223), tensor(-0.5777))

In [175]:
torch.mean(V), torch.max(V), torch.min(V)

(tensor(1.1809e-05), tensor(0.3573), tensor(-0.5567))

In [123]:
labels_counts = torch.tensor([(y==i).sum() for i in range(len(labels))])
weights = 1 - labels_counts / len(X)    

In [85]:
#weights = torch.tensor([0.1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [168]:
model = nn.Sequential(
    nn.Linear(1408, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [169]:
sum(param.numel() for param in model.parameters())

886922

In [170]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Sequential(
  (0): Linear(in_features=1408, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=10, bias=True)
)

In [171]:
weights = weights.to(device)
loss_fn = nn.CrossEntropyLoss(weights)  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [181]:
n_epochs = 30
batch_size = 128
for epoch in range(n_epochs):
    acc = 0
    perm = torch.randperm(len(X))
    for i in range(0, len(X), batch_size):
        Xbatch = X[perm[i:i+batch_size]]
        Xbatch = Xbatch.to(device)
        y_pred = model(Xbatch)
        ybatch = y[perm[i:i+batch_size]]
        ybatch = ybatch.type(torch.LongTensor)
        ybatch = ybatch.to(device)
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc += get_acc(y_pred, ybatch, return_sum=True)
    val_pred = evaluate(V)
        
    print(f'Finished epoch {epoch}, latest loss {loss}, acc {acc/len(X)} val_acc {get_acc(val_pred, y_val)}')

Finished epoch 0, latest loss 0.48803332448005676, acc 0.8604374527931213 val_acc 0.6837040781974792
Finished epoch 1, latest loss 0.5362734794616699, acc 0.8619195818901062 val_acc 0.6826694011688232
Finished epoch 2, latest loss 0.6099376678466797, acc 0.8629425764083862 val_acc 0.67475426197052
Finished epoch 3, latest loss 0.46636611223220825, acc 0.8645455241203308 val_acc 0.6749094724655151
Finished epoch 4, latest loss 0.3988199830055237, acc 0.8654879927635193 val_acc 0.6695291996002197
Finished epoch 5, latest loss 0.4560309648513794, acc 0.8663579225540161 val_acc 0.6674081683158875
Finished epoch 6, latest loss 0.48931875824928284, acc 0.8678159117698669 val_acc 0.6669425964355469
Finished epoch 7, latest loss 0.29093995690345764, acc 0.8689113855361938 val_acc 0.6608898043632507
Finished epoch 8, latest loss 0.2862897217273712, acc 0.8702002167701721 val_acc 0.6582514047622681
Finished epoch 9, latest loss 0.3907463848590851, acc 0.8709332346916199 val_acc 0.659079134464263

In [182]:
get_acc_per_lbl(val_pred, y_val)

0, presc 0.76, recall 0.86, f1 0.81	 total 12895 Normal
1, presc 0.28, recall 0.32, f1 0.30	 total 646 Abuse
2, presc 0.61, recall 0.06, f1 0.11	 total 1321 Arrest
3, presc 0.15, recall 0.88, f1 0.26	 total 106 Arson
4, presc 0.25, recall 0.14, f1 0.18	 total 1767 Burglary
5, presc 0.56, recall 0.04, f1 0.07	 total 388 Explosion
6, presc 0.47, recall 0.15, f1 0.23	 total 1132 Fighting
7, presc 0.00, recall 0.00, f1 nan	 total 12 RoadAccidents
8, presc 0.00, recall 0.05, f1 0.00	 total 44 Shooting
9, presc 0.42, recall 0.28, f1 0.34	 total 1019 Vandalism


In [183]:
get_acc(val_pred, y_val)

tensor(0.6338)

In [184]:
X_pred = evaluate(X)

In [185]:
get_acc_per_lbl(X_pred, y)

0, presc 0.95, recall 0.89, f1 0.92	 total 84684 Normal
1, presc 0.60, recall 0.73, f1 0.66	 total 2593 Abuse
2, presc 0.86, recall 0.98, f1 0.92	 total 10454 Arrest
3, presc 0.83, recall 0.96, f1 0.89	 total 4984 Arson
4, presc 0.84, recall 0.96, f1 0.90	 total 8211 Burglary
5, presc 0.72, recall 0.86, f1 0.78	 total 2138 Explosion
6, presc 0.69, recall 0.90, f1 0.78	 total 6178 Fighting
7, presc 0.47, recall 0.03, f1 0.06	 total 689 RoadAccidents
8, presc 0.55, recall 0.41, f1 0.47	 total 1568 Shooting
9, presc 0.71, recall 0.85, f1 0.78	 total 2646 Vandalism


In [186]:
get_acc(X_pred, y)

tensor(0.8884)

In [187]:
torch.save(model.state_dict(), "model_image_features")